# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [10]:
df.action.unique()

array(['view', 'click'], dtype=object)

In [24]:
num_of_viewers = set(df[df['action']== 'view']['id'].unique())
num_of_clickers = set(df[df['action']=='click']['id'].unique())

In [29]:
viewers_no_click = len(num_of_viewers - num_of_clickers)
viewers_no_click

4468

In [31]:
clickers_no_view = len(num_of_clickers - num_of_viewers)
clickers_no_view
# No clickers who didn't view

0

In [32]:
# Checking for overlaps
exp = set(df[df['group']=='experiment']['id'].unique())
ctrl = set(df[df['group']=='control']['id'].unique())
intersect = len(exp&ctrl)
intersect
#No overlaps

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [33]:
#Your code here
df['count'] = 1
df

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
...,...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view,1
8184,2017-01-18 09:42:12.844575,755912,experiment,view,1
8185,2017-01-18 10:01:09.026482,458115,experiment,view,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1


In [57]:
# Creating a pivot table
ctrl_table = df[df['group'] == 'control'].pivot('id','action','count')
ctrl_table = ctrl_table.fillna(0)
exp_table = df[df['group'] == 'experiment'].pivot('id','action','count')
exp_table = exp_table.fillna(0)
exp_table

C:\Users\hp\AppData\Local\Temp\ipykernel_8876\1934206810.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  ctrl_table = df[df['group'] == 'control'].pivot('id','action','count')
C:\Users\hp\AppData\Local\Temp\ipykernel_8876\1934206810.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  exp_table = df[df['group'] == 'experiment'].pivot('id','action','count')


action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [58]:
import numpy as np
import scipy.stats as stats

results = stats.ttest_ind(ctrl_table.click,exp_table.click,equal_var = False)
p = results[1]/2
p

0.004466402814337102

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [59]:
#Your code here
n = len(exp_table)
p = ctrl_table.click.mean()
expected_num_of_clicks_for_the_exp_group = n*p
expected_num_of_clicks_for_the_exp_group

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [61]:
#Your code here
var = n*p*(1-p)
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [63]:
#Your code here
num_expr_clicks = exp_table.click.sum()
z_score = (num_expr_clicks - expected_num_of_clicks_for_the_expr_group)/std
p_val = stats.norm.sf(np.abs(z_score))
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
"""
The p_values are different but, both reject the null hypothesis since they are lower than our alpha of 0.05.
There is enough evidence to say that the experimental page is significantly effective 
than the current page. 
"""

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.